In [7]:
from models import SGCRes
from dgl.data import CoraGraphDataset, CiteseerGraphDataset, PubmedGraphDataset
import torch.nn as nn
import torch
import torch.nn.functional as F
import dgl
from time import perf_counter

In [8]:
device = 'cpu'
activation = nn.ReLU()
epochs = 50
batch_size = 10000
lr = 0.02
loss_fn = nn.CrossEntropyLoss()
weight_decay = 5e-4

In [9]:
data = PubmedGraphDataset()
g = data[0]
g = g.to(device)
features = g.ndata['feat']
labels = g.ndata['label']
train_mask = g.ndata['train_mask']
val_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']
in_feats = features.shape[-1]
n_classes = data.num_labels
n_edges = g.number_of_edges()
g = dgl.remove_self_loop(g)
g = dgl.add_self_loop(g)

  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


d:\Files\Coding\ml\EfficientGNN\venv\lib\site-packages\dgl\data\utils.py:288: UserWarning: Property dataset.num_labels will be deprecated, please use dataset.num_classes instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))


In [10]:
model = SGCRes(in_feats, n_classes, 2, 2)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [11]:
def train(model, g, features, labels, train_mask, val_mask, test_mask):
    t = perf_counter()
    for epoch in range(epochs):
        model.train()
        logits = model(g, features)
        loss = loss_fn(logits[train_mask], labels[train_mask])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        logits = model(g, features)
        train_acc = torch.sum(logits[train_mask].argmax(1) == labels[train_mask]).item() / train_mask.sum().item()
        val_acc = torch.sum(logits[val_mask].argmax(1) == labels[val_mask]).item() / val_mask.sum().item()
        test_acc = torch.sum(logits[test_mask].argmax(1) == labels[test_mask]).item() / test_mask.sum().item()
        print(f'Epoch {epoch + 1:02d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

    training_time = perf_counter()-t
    print(f'Training time: {training_time:.4f}s')

In [12]:
train(model, g, features, labels, train_mask, val_mask, test_mask)

Epoch 01, Loss: 1.3579, Train: 0.5333, Val: 0.4620, Test: 0.4590
Epoch 02, Loss: 1.1541, Train: 0.5833, Val: 0.5300, Test: 0.5200
Epoch 03, Loss: 0.9364, Train: 0.7333, Val: 0.6620, Test: 0.6520
Epoch 04, Loss: 0.8015, Train: 0.7667, Val: 0.6920, Test: 0.6870
Epoch 05, Loss: 0.7459, Train: 0.8167, Val: 0.6900, Test: 0.6880
Epoch 06, Loss: 0.6463, Train: 0.8667, Val: 0.6900, Test: 0.6850
Epoch 07, Loss: 0.5227, Train: 0.8833, Val: 0.7020, Test: 0.6940
Epoch 08, Loss: 0.4453, Train: 0.8500, Val: 0.6720, Test: 0.6680
Epoch 09, Loss: 0.4412, Train: 0.8833, Val: 0.6680, Test: 0.6670
Epoch 10, Loss: 0.4257, Train: 0.9000, Val: 0.6860, Test: 0.6750
Epoch 11, Loss: 0.3807, Train: 0.9000, Val: 0.6960, Test: 0.6930
Epoch 12, Loss: 0.3432, Train: 0.9167, Val: 0.7100, Test: 0.7050
Epoch 13, Loss: 0.3174, Train: 0.9167, Val: 0.7200, Test: 0.7230
Epoch 14, Loss: 0.2983, Train: 0.9500, Val: 0.7440, Test: 0.7330
Epoch 15, Loss: 0.2835, Train: 0.9500, Val: 0.7540, Test: 0.7380
Epoch 16, Loss: 0.2712, T